In [52]:
from __future__ import division
import argparse
import pandas as pd
import numpy as np
import re  # Used to split text into sentences
import string  # Used to get all punctuation
import pickle  # Used to save and load embeddings
import logging  # Used to save steps in a text file instead of printing them
import tqdm  # Used to time the training
from scipy.special import expit  # Used to compute the gradient


__authors__ = ['Driss Debbagh-Nour','Mehdi Mikou','Soufiane Hadji', 'Mohamed Aymane Benayada']
__emails__  = ['driss.debbagh-nour@student.ecp.fr','mehdi.mikou@student.ecp.fr',
               'soufiane.hadji@student.ecp.fr', "mohamed-aymane.benayada@student.ecp.fr"]


logging.basicConfig(filename='test_log.log',level=logging.INFO,\
      format='%(asctime)s,%(msecs)d %(levelname)-8s [%(filename)s:%(lineno)d] %(message)s')


def concat_text(file):
    '''
    Used for documents which contain a long text rather than multiple sentences.
    The functions concatenate the whole text and removes quotes
    '''
    all_text_list = []
    with open(file) as f:
        for l in f:
            l = l[:-1] #We remove the line jumping
            all_text_list.append(l)
    concat_text = ''.join(all_text_list)
    concat_text = concat_text.replace('"', '')
    concat_text = concat_text.replace("\'", '')
    
    return concat_text


def text2sentences(file, only_sentences=True):
    """
    Split words while removing all punctuation, stopwords and non-alpha words while keeping contractions together
    2 types of input: 
    - If it's a whole text: Split text into sentences using punctuation (. ? ! ;) after concatenating all text
    - If it's a bunch of sentences: Split the document using \n 
    Then split sentences by whitespace
    
    Returns the tokenization of our file
    """
    
    final_sentences = []
    
    # Load stop words
    #stop_words = stopwords.words('english')
    #stop_words.append("")
    
    # Differentiate the two types of documents
    if not only_sentences:
        concatenated_file = concat_text(file)
        sentences = re.split(r'[.?!;]', concatenated_file)
    else:
        sentences = [line.rstrip('\n') for line in open(file, encoding="utf8")]
    
    # Tokenize our document
    for sentence in sentences:
        words_list = sentence.lower().split()
        ponctuation_remover = str.maketrans('', '', string.punctuation)
        stripped = [w.translate(ponctuation_remover) for w in words_list 
                    if w not in string.punctuation]
        final_sentence = [word for word in stripped if word.isalpha()]
        final_sentences.append(final_sentence)

    return final_sentences


def count_words(list_words):
    """
    Take a list of words and return a dictionary with words as keys and their occurence as values
    """
    dict_occurences = {}
    for word in list_words:
        try:
            dict_occurences[word] += 1
        except KeyError:
            dict_occurences[word] = 1
    return dict_occurences
            
    
def rare_word_pruning(sentences, min_count):
    """
    Remove words that occures less than min_count time
    """
    words = [word for sentence in sentences for word in sentence]
    dict_occurences = count_words(words)
    new_sentences = []
    for sentence in sentences:
        new_sentence = []
        for word in sentence:
            if dict_occurences[word] >= min_count:
                new_sentence.append(word)
        new_sentences.append(new_sentence)
    return new_sentences

        
# def high_and_low_frequency_pruning(sentences, min_count, max_count_ratio):
#     """
#     Remove words that occures less than min_count and more than max_count times
#     """
#     words = [word for sentence in sentences for word in sentence]
#     max_count = int(len(words) * max_count_ratio)
#     dict_occurences = count_words(words)
#     new_sentences = []
#     for sentence in sentences:
#         new_sentence = []
#         for word in sentence:
#             if min_count<= dict_occurences[word] <= max_count:
#                 new_sentence.append(word)
#         new_sentences.append(new_sentence)
#     return new_sentences


def get_positive_pairs(processed_sentences, winSize):
    """
    Get Pairs of words that co-occur (in the window delimited by winSize): Positive examples
    """
    # List of positive pairs to return
    positive_pairs = []
    
    # Initialize dictionaries that will allow to move from strings to their indexes
    words_voc = {}  
    context_voc = {}
    
    # Initialize indexes for words and contexts
    indexer_words = 0
    indexer_context = 0
    
    for sentence in processed_sentences:
        for i, word in enumerate(sentence):
            if word not in words_voc:
                words_voc[word] = indexer_words
                indexer_words += 1
            
            for j in range(max(0, i - winSize//2), min(i + winSize//2 + 1, len(sentence))):  # Be careful of edges
                if i != j:  # word != context
                    context = sentence[j]
                    if context not in context_voc:
                        context_voc[context] = indexer_context
                        indexer_context += 1
                    positive_pairs.append((words_voc[word], context_voc[context]))
                    
    return positive_pairs, words_voc, context_voc


def get_negative_pairs(positive_pairs, negativeRate):
    """
    Get Pairs of words that don't co-occur: Negative examples 
    Size: negativeRate * size(positive_pairs)
    """
    # List of negative pairs to return
    negative_pairs = []
    nbr_positive_pairs = len(positive_pairs)
    
    for p_pair in positive_pairs:
        word_index = p_pair[0]  # target word
        for _ in range(negativeRate):
            pair_index = np.random.randint(nbr_positive_pairs)  # Random index for a pair (can be improved)
            negative_context = positive_pairs[pair_index][1]  # Get the random pair context index
            negative_pairs.append((word_index, negative_context))
            
    return negative_pairs


def gradient(theta, nEmbed, positive_pairs, negative_pairs, nb_words, nb_contexts):
    """
    Compute gradient at init_theta for positive and negative pairs
    """
    # Initialize gradient
    grad = np.zeros(len(theta))
    
    # Embedding matrix of target words
    words_matrix = theta[: nEmbed * nb_words].reshape(nb_words, nEmbed)
    
    # Embedding matrix of contextes
    contexts_matrix = theta[nEmbed * nb_words:].reshape(nb_contexts, nEmbed)
    
    logging.info("Compute gradient")
    
    # Positive pairs
    logging.info("Positive pairs...")
    for p_pair in positive_pairs:
        
        # Get indexes of (word, context)
        word_index = p_pair[0]
        context_index = p_pair[1]
        
        # Get the actual embedding of the word and its context
        word = words_matrix[word_index]
        context = contexts_matrix[context_index]

        # We compute the derivative of the formula given by 'Yoav Goldberg' and 'Omer Levy'
        df_word = context * expit(-word.dot(context))
        df_context = word * expit(-word.dot(context))
        
        # We actualize the gradient of the word and its context
        grad[word_index * nEmbed: (word_index + 1) * nEmbed] += df_word
        grad[(nb_words + context_index) * nEmbed: (nb_words + context_index + 1) * nEmbed] += df_context
    logging.info("Done")
    
    # Negative pairs
    logging.info("Negative pairs...")
    for n_pair in negative_pairs:
        
        # Get indexes of (word, negative context)
        word_index = n_pair[0]
        context_index = n_pair[1]
        
        # Get the actual embedding of the word and its context
        word = words_matrix[word_index]
        context = contexts_matrix[context_index]
        
        # We compute the derivative of the formula given by 'Yoav Goldberg' and 'Omer Levy'
        df_word = -context * expit(word.dot(context))
        df_context = -word * expit(word.dot(context))
        
        # We actualize the gradient of the word and its negative context
        grad[word_index * nEmbed: (word_index + 1) * nEmbed] += df_word
        grad[(nb_words + context_index) * nEmbed: (nb_words + context_index + 1) * nEmbed] += df_context
    logging.info("Done")
    
    return grad

        
def loadPairs(path):
    data = pd.read_csv(path, delimiter='\t')
    pairs = zip(data['word1'], data['word2'], data['similarity'])
    return pairs


class SkipGram:
    def __init__(self, sentences, nEmbed=100, negativeRate=5, winSize=7, minCount=5):
        # Preprocessing the sentences
        # Remove rare words
        print("1. Processing sentences...")
        processed_sentences= rare_word_pruning(sentences, minCount)
        print("Done\n")
        
        # Generate positive and negative pairs
        print("2. Generating positive pairs...")
        self.positive_pairs, self.words_voc, self.context_voc = get_positive_pairs(processed_sentences, winSize)
        print("Done\n")
        
        print("3. Generating negative samples...")
        # Generate negative samples
        self.negative_pairs = get_negative_pairs(self.positive_pairs, negativeRate)
        print("Done")

    def train(self, learning_rate=0.01, epochs=5, batchsize=500, nEmbed=100, negativeRate=5):
        """Create W matrix containing the embeddings of all words"""
        
        # Get nb of words, nb of contexts, nb of pairs
        nb_words = len(list(self.words_voc.keys()))
        nb_contexts = len(list(self.context_voc.keys()))
        nb_pairs = len(self.positive_pairs)
  
        # Initialize theta: vector of parameters
        nb_param = nEmbed * (nb_words + nb_contexts)  # Number of parameters
        theta = np.random.random(nb_param) * 1e-5

        # Compute Stochastic Gradient
        print("TRAINING: epochs: {}, learning_rate: {}, batch size: {}".format(epochs, learning_rate, batchsize))
        logging.info("TRAINING: epochs: {}, learning_rate: {}, batch size: {}".format(epochs, learning_rate, batchsize))
        for epoch in range(epochs):
            print("Epoch {}/{}".format(epoch+1, epochs))
            logging.info("Epoch {}/{}".format(epoch+1, epochs))
            
            # We update theta after computing the gradient of each batch (which size is batchsize)
            for batch_number in tqdm.tqdm(range(nb_pairs // batchsize)):
                logging.info("batch_number {}/{}".format(batch_number+1, nb_pairs // batchsize))
                batch_begin = batch_number * batchsize
                batch_end = min((batch_number + 1) * batchsize, nb_pairs)
                batch_positive = self.positive_pairs[batch_begin: batch_end]
                batch_negative = self.negative_pairs[negativeRate * batch_begin: negativeRate * batch_end]
                
                # Compute the gradient at theta
                grad = gradient(theta, nEmbed, batch_positive, batch_negative, nb_words, nb_contexts)
                
                # Actualize theta (since we want to maximize the 'loss', we add grad)
                theta = theta + learning_rate*grad
                
            logging.info(theta)

        self.theta = theta
        
        # Matrix of embeddings
        self.W = theta[:nEmbed * nb_words].reshape(nb_words, nEmbed)

        
    def save(self, path):
        """Save in binary Theta and W"""
        with open(path + '\\theta', 'wb') as fichier:
            mon_pickler = pickle.Pickler(fichier)
            mon_pickler.dump(self.theta)
        with open(path + '\\W', 'wb') as fichier:
            mon_pickler = pickle.Pickler(fichier)
            mon_pickler.dump(self.W)        
        with open(path + '\\words_voc', 'wb') as fichier:
            mon_pickler = pickle.Pickler(fichier)
            mon_pickler.dump(self.words_voc)
            
            
    def similarity(self, word1, word2, words_voc, W):
        """
        computes similiarity between the two words. unknown words are mapped to one common vector
        :param word1: 1st word
        :param word2: 2nd word
        :param words_voc: dictionary of words kept from original document as keys and their index as value 
        :param W: matrix of embeddings
        :return: a float \in [0,1] indicating the similarity (the higher the more similar)
        """
        nEmbed = W.shape[1]
        
        # For words that aren't in the training set, we considere them as the same "OOV" 
        # and give them the same vector with low values
        default_embd = np.ones(nEmbed) * 0.01
        
        # Get word_1 vector
        if word1 in words_voc:
            idx_word1 = words_voc[word1]
            embd_word1 = W[idx_word1]
        else:
            print("Out of Vocabulary:", str(word1))
            embd_word1 = default_embd
        
        # Get word_2 vector
        if word2 in words_voc:
            idx_word2 = words_voc[word2]
            embd_word2 = W[idx_word2]
        else:
            print("Out of Vocabulary:", str(word2))
            embd_word2 = default_embd
        
        # Compute the cosine distance
        return abs(embd_word1.dot(embd_word2) / (np.linalg.norm(embd_word1) * np.linalg.norm(embd_word2)))

    
    def K_most_similar(self, word, K, words_voc, W):
        dict_words_similarity = {}
        for elt in words_voc:
            dict_words_similarity[elt] = self.similarity(word, elt, words_voc, W)
            
        ranked_similar_words = sorted(dict_words_similarity, key=dict_words_similarity.get, reverse=True)
        print("Similar words for", word, ":")
        for i in range(1, K + 1):
            print("-", ranked_similar_words[i], ':', dict_words_similarity[ranked_similar_words[i]])
    
    
    @staticmethod
    def load(path):
        with open(path + '\\W', 'rb') as fichier:
            my_depickler = pickle.Unpickler(fichier)
            W = my_depickler.load()
        with open(path + '\\words_voc', 'rb') as fichier:
            my_depickler = pickle.Unpickler(fichier)
            words_voc = my_depickler.load()
                  
        return W, words_voc


if __name__ == '__main__':

    parser = argparse.ArgumentParser()
    parser.add_argument('--text', help="news.en-00001-of-00100.txt", required=True)
    parser.add_argument('--model', help='test', required=True)
    parser.add_argument('--test', help='test', action='store_true')

    opts = parser.parse_args()

    if not opts.test:
        sentences = text2sentences(opts.text)
        sg = SkipGram(sentences)
        sg.train()
        sg.save(opts.model)

    else:
        pairs = loadPairs(opts.text)
        W, words_voc = SkipGram.load(opts.model)
        
        for a,b,_ in pairs:
            print(sg.similarity(a,b, words_voc, W))



### Load data set

In [53]:
sentences = text2sentences("news.en-00001-of-00100.txt", only_sentences=True)

### Information about data set

In [54]:
len(sentences)

306068

In [57]:
crop_sentences = sentences[:10000]
len(crop_sentences)
# print(crop_sentences[])

10000

In [58]:
counter = 0
for sent in crop_sentences:
    for word in sent:
        counter += 1
counter

219305

In [74]:
monskip = SkipGram(crop_sentences, nEmbed=300, negativeRate=5, winSize=7, minCount=5)

1. Processing sentences...
Done

2. Generating positive pairs...
Done

3. Generating negative samples...
Done


In [75]:
print(len(monskip.positive_pairs))
print(len(monskip.negative_pairs))
print(len(monskip.words_voc))
print(len(monskip.context_voc))

1020658
5103290
4901
4901


### Train the model

In [76]:
monskip.train(learning_rate=0.01, epochs=5, batchsize=500, nEmbed=300, negativeRate=5)

TRAINING: #epochs: 5, learning_rate: 0.01, batch size: 500
Epoch 1/5


100%|██████████████████████████████████████| 2041/2041 [07:48<00:00,  4.46it/s]


(470.14s)
Epoch 2/5


100%|██████████████████████████████████████| 2041/2041 [07:40<00:00,  4.52it/s]


(460.41s)
Epoch 3/5


...

(458.43s)
Epoch 4/5


100%|██████████████████████████████████████| 2041/2041 [07:48<00:00,  4.49it/s]


(468.34s)
Epoch 5/5


100%|██████████████████████████████████████| 2041/2041 [08:09<00:00,  4.13it/s]


(489.89s)


## Save the model

In [78]:
monskip.save(r"gridsearch\10k\dimension_300")
W, words_voc = monskip.load(r"gridsearch\10k\dimension_300")

In [81]:
# print(monskip.similarity("obama","president"))
# monskip.similarity("achilles", "driss", words_voc, W)
monskip.K_most_similar("president", 10, words_voc, W)

Similar words for president :
- chief : 0.9330720967728672
- prime : 0.9213076605844395
- executive : 0.9079792179962911
- minister : 0.9070468374519342
- obama : 0.9037875920805644
- secretary : 0.8870692732115408
- state : 0.8798835145992542
- director : 0.8783563506443371
- john : 0.8777978122809695
- former : 0.8775287728786312


In [16]:
monskip.similarity("woman", "man", words_voc, W)

0.9003719041046656

In [24]:
monskip.similarity("woman", "girl", words_voc, W)

0.9140292408030202

In [18]:
monskip.similarity("woman", "bicycle", words_voc, W)

0.5525813805008297

In [32]:
monskip.similarity("woman", "grizzly", words_voc, W)

Out of Vocabulary: grizzly


0.2520814392860999

## Grid Search

In [69]:
winSize_range  = [3, 9, 11]
sentences = text2sentences("news.en-00001-of-00100.txt", only_sentences=True)
crop_sentences = sentences[:10000]
for winSize_value in winSize_range:
    monskip = SkipGram(crop_sentences, nEmbed=100, negativeRate=5, winSize=winSize_value, minCount=5)
    monskip.train(learning_rate=0.01, epochs=5, batchsize=500, nEmbed=100, negativeRate=5)
    monskip.save(r"C:\Users\Driss Debbagh\Desktop\Cours 3A\Natural_Language_Processing\SkipGram\gridsearch\10k\winsize_" + str(winSize_value))
    
negativerate_range = [2, 8]
sentences = text2sentences("news.en-00001-of-00100.txt", only_sentences=True)
crop_sentences = sentences[:10000]
for negativerate_value in negativerate_range:
    monskip = SkipGram(crop_sentences, nEmbed=100, negativeRate=negativerate_value, winSize=5, minCount=5)
    monskip.train(learning_rate=0.01, epochs=5, batchsize=500, nEmbed=100, negativeRate=negativerate_value)
    monskip.save(r"C:\Users\Driss Debbagh\Desktop\Cours 3A\Natural_Language_Processing\SkipGram\gridsearch\10k\negativerate_" + str(negativerate_value))
   

1. Processing sentences...
Done

2. Generating positive pairs...
Done

3. Generating negative samples...
Done
TRAINING: #epochs: 5, learning_rate: 0.01, batch size: 500
Epoch 1/5


100%|████████████████████████████████████████| 720/720 [02:14<00:00,  5.28it/s]


(134.08s)
Epoch 2/5


100%|████████████████████████████████████████| 720/720 [02:13<00:00,  5.54it/s]


(133.99s)
Epoch 3/5


...

(371.32s)
Epoch 4/5


100%|██████████████████████████████████████| 1400/1400 [06:00<00:00,  3.75it/s]


(360.79s)
Epoch 5/5


100%|██████████████████████████████████████| 1400/1400 [06:13<00:00,  3.80it/s]


(373.89s)


### Testing each value of the hyperparameters

#### Window Size

In [72]:
winSize_range  = [3, 5, 7, 9, 11]
for window_size in winSize_range:
    monskip = SkipGram(crop_sentences, nEmbed=100, negativeRate=5, winSize=window_size, minCount=5)
    W, words_voc = monskip.load(r"C:\Users\Driss Debbagh\Desktop\Cours 3A\Natural_Language_Processing\SkipGram\gridsearch\10k\winsize_" + str(window_size))
    print("\nWindow size:", window_size)
    monskip.K_most_similar("president", 10, words_voc, W)
    print("\n---------------------------------\n")

1. Processing sentences...
Done

2. Generating positive pairs...
Done

3. Generating negative samples...
Done

Window size: 3
Similar words for president :
- director : 0.9792246591856569
- california : 0.97649384551502
- chief : 0.9746639693673325
- chairman : 0.9742586167911036
- bank : 0.9715236174124944
- america : 0.9706534470582433
- leader : 0.96962433136895
- london : 0.967842092831337
- office : 0.9670202297599934
- life : 0.9665494499288213

---------------------------------

1. Processing sentences...
Done

2. Generating positive pairs...
Done

3. Generating negative samples...
Done

Window size: 5
Similar words for president :
- chief : 0.9593564101602977
- secretary : 0.9494704742934456
- executive : 0.9461626633693925
- prime : 0.9300461180738658
- angeles : 0.9286220659735738
- deputy : 0.9242985502729822
- director : 0.9224185316975538
- los : 0.920480357299054
- former : 0.9204484530722087
- minister : 0.917344428830076

---------------------------------

1. Processing

#### Negative rate

In [73]:
negativerate_range = [2, 5, 8]
for negative_rate in negativerate_range:
    monskip = SkipGram(crop_sentences, nEmbed=100, negativeRate=negative_rate, winSize=window_size, minCount=5)
    W, words_voc = monskip.load(r"C:\Users\Driss Debbagh\Desktop\Cours 3A\Natural_Language_Processing\SkipGram\gridsearch\10k\negativerate_" + str(negative_rate))
    print("\nNegative rate:", negative_rate)
    monskip.K_most_similar("president", 10, words_voc, W)
    print("\n---------------------------------\n")
    

1. Processing sentences...
Done

2. Generating positive pairs...
Done

3. Generating negative samples...
Done

Negative rate: 2
Similar words for president :
- chief : 0.9718132956742079
- korea : 0.9642909245524272
- foreign : 0.9552330469471004
- secretary : 0.954838870589087
- executive : 0.9546695762660887
- los : 0.9540462476995708
- saturday : 0.9527672005514987
- south : 0.9483294769950554
- director : 0.9451422045123068
- smoking : 0.9419185743388031

---------------------------------

1. Processing sentences...
Done

2. Generating positive pairs...
Done

3. Generating negative samples...
Done

Negative rate: 5
Similar words for president :
- chief : 0.9593564101602977
- secretary : 0.9494704742934456
- executive : 0.9461626633693925
- prime : 0.9300461180738658
- angeles : 0.9286220659735738
- deputy : 0.9242985502729822
- director : 0.9224185316975538
- los : 0.920480357299054
- former : 0.9204484530722087
- minister : 0.917344428830076

---------------------------------

1. 